In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()


# For using code without restarting.
%load_ext autoreload
%autoreload 
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml inference_config

log:
    root: '?'
    log_interval: 10 
    log_image_stats: '?' 
    log_pixel_stats: '?' 
    show_examples: '?' 

experiment:
    seed: 42

data:
    split: cal 
    input_type: image
    # slicing: dense_full 
    # input_type: volume 
    preload: '?' 

dataloader:
    batch_size: 1 
    num_workers: 0
    pin_memory: False 

calibration:
    num_bins: 15
    neighborhood_width: 3
    square_diff: False 

<IPython.core.display.Javascript object>

In [3]:
%%yaml model_cfg

model:
    pretrained_exp_root : '?' 
    checkpoint: '?' 
    ensemble: '?' 
    ensemble_cfg: '?'
    ensemble_w_metric: '?'
    pretrained_select_metric: "val-dice_score"

<IPython.core.display.Javascript object>

In [4]:
%%yaml metrics_cfg 

qual_metrics:
    - Dice:
        _fn: ese.experiment.metrics.dice_score
        from_logits: True
        batch_reduction: 'mean' 
        ignore_empty_labels: True # False for WMH only
        ignore_index: 0 # Ignore background class when reporting.
        metric_type: quality
    - HD95:
        _fn: ese.experiment.metrics.hd95
        from_logits: True
        batch_reduction: 'mean' 
        ignore_empty_labels: True # False for WMH only
        ignore_index: 0 # Ignore background class when reporting.
        metric_type: quality

image_cal_metrics:
    - Image_ECE:
        _fn: ese.experiment.metrics.ece.image_ece_loss
    - Image_Edge-ECE:
        _fn: ese.experiment.metrics.ece.image_edge_ece_loss
    - Image_CW-ECE:
        _fn: ese.experiment.metrics.ece.image_cw_ece_loss
    - Image_ELM:
        _fn: ese.experiment.metrics.elm.image_elm_loss
    - Image_Edge-ELM:
        _fn: ese.experiment.metrics.elm.image_edge_elm_loss
    # - Image_Foreground-ECE:
    #     _fn: ese.experiment.metrics.ece.image_ece_loss
    #     ignore_index: 0
    # - Image_Foreground-Edge-ECE:
    #     _fn: ese.experiment.metrics.ece.image_edge_ece_loss
    #     ignore_index: 0
    # - Image_Foreground-CW-ECE:
    #     _fn: ese.experiment.metrics.ece.image_cw_ece_loss
    #     ignore_index: 0
    # - Image_Foreground-ELM:
    #     _fn: ese.experiment.metrics.elm.image_elm_loss
    #     ignore_index: 0
    # - Image_Foreground-Edge-ELM:
    #     _fn: ese.experiment.metrics.elm.image_edge_elm_loss
    #     ignore_index: 0

# Define global metrics when you want to verify that these are the same as the image
# based metrics, per image.
global_cal_metrics:
    - ECE:
        _fn: ese.experiment.metrics.ece.ece_loss
    - Edge-ECE:
        _fn: ese.experiment.metrics.ece.edge_ece_loss
    - CW-ECE:
        _fn: ese.experiment.metrics.ece.cw_ece_loss
    - ELM:
        _fn: ese.experiment.metrics.elm.elm_loss
    - Edge-ELM:
        _fn: ese.experiment.metrics.elm.edge_elm_loss
    # - Foreground-ECE:
    #     _fn: ese.experiment.metrics.ece.ece_loss
    #     ignore_index: 0
    # - Foreground-Edge-ECE:
    #     _fn: ese.experiment.metrics.ece.edge_ece_loss
    #     ignore_index: 0
    # - Foreground-CW-ECE:
    #     _fn: ese.experiment.metrics.ece.cw_ece_loss
    #     ignore_index: 0
    # - Foreground-ELM:
    #     _fn: ese.experiment.metrics.elm.elm_loss
    #     ignore_index: 0
    # - Foreground-Edge-ELM:
    #     _fn: ese.experiment.metrics.elm.edge_elm_loss
    #     ignore_index: 0

<IPython.core.display.Javascript object>

In [5]:
from ionpy.util import dict_product, Config
from ionpy.util.config import check_missing
from ese.scripts.utils import gather_exp_paths


def get_ese_inference_configs(
        calibrator, 
        do_ensemble: bool, 
        dataset: str,
        ensemble_w_metric: str,
        exp_group_string: str,
        base_ensemble_group: str,
        calibration_group: str,
        log_image_stats: bool = True,
        log_pixel_stats: bool = True,
        preload: bool = False, 
        show_examples: bool = False,
        ):
    # Define the paths for the uncailbrated networks.
    ##################################################
    if calibrator == "None":
        calibrator = "Uncalibrated"
        ensemble_root = f"/storage/vbutoi/scratch/ESE/training/{base_ensemble_group}"
        checkpoint = "max-val-dice_score" 
    # Define the paths for the calibrated networks.
    ##################################################
    else:
        ensemble_root = f"/storage/vbutoi/scratch/ESE/calibration/{calibration_group}/Individual_{calibrator}"
        checkpoint = "min-val-ece_loss"

    # Set a few things that will be consistent for all runs.
    ##################################################
    default_config_options = {
        'model.checkpoint': [checkpoint],
        'data.preload': [preload],
        'log.show_examples': [show_examples],
        'log.log_image_stats': [log_image_stats],
        'log.log_pixel_stats': [log_pixel_stats]
    }

    exp_root = root / "inference" / exp_group_string
    # Make presets for the different runnning configurations.
    ##################################################
    # If you want to run inference on ensembles, use this.
    if do_ensemble:
        advanced_args = {
            'log.root': [str(exp_root / f"{dataset}_Ensemble_{calibrator}")],
            'model.pretrained_exp_root': [ensemble_root],
            'model.ensemble': [True],
            'model.ensemble_w_metric': [ensemble_w_metric],
            'model.ensemble_cfg': [
                ('mean', 'logits'), 
                ('mean', 'probs'), 
                ('product', 'probs')
           ]
        }
    # If you want to run inference on individual networks, use this.
    else:
        advanced_args = {
            'log.root': [str(exp_root / f"{dataset}_Individual_{calibrator}")],
            'model.pretrained_exp_root': gather_exp_paths(ensemble_root), # Note this is a list of train exp paths.
            'model.ensemble': [False],
            'model.ensemble_cfg': [None],
            'model.ensemble_w_metric': [None]
        }
    # Combine the default and advanced arguments.
    default_config_options.update(advanced_args)
    log_root = default_config_options['log.root'][0]
    return default_config_options, log_root

In [6]:
# Calibrators
# - None
# - Vanilla 
# - TempScaling
# - VectorScaling
# - DirichletScaling
# - LTS
# - NectarScaling
# - ConstrainedNS 

# Ensemble Weightings
# - None
# - val loss
# - val dice
# - val ece
# - val edge ece
# - val elm

# Calibration groups
# - 01_18_24_WMH_All_Calibrators (Basic cross entropy)

# Inference groups
# - 01_18_24_All_Metrics (Basic cross entropy)

from datetime import datetime
# Get today's date
today_date = datetime.now()
# Format the date as MM_DD_YY
formatted_date = today_date.strftime("%m_%d_%y")

# Get the configs for the different runs.
dataset_options, log_root = get_ese_inference_configs(
    calibrator='None', 
    do_ensemble=False, 
    ensemble_w_metric='None',
    dataset='CityScapes',
    base_ensemble_group='01_25_24_CityScapes_Dice',
    calibration_group='01_26_24_CityScapes_Ensemble',
    exp_group_string='01_27_24_CityScapes_Analysis',
    show_examples=False,
    preload=False
)

In [7]:
%%yaml aug_cfg

# Used for CityScapes.
augmentations:
    - Resize:
        height: 256
        width: 512

<IPython.core.display.Javascript object>

In [8]:
# Build the configs.
##################################################
base_cfg = Config(inference_config).update([model_cfg, metrics_cfg, aug_cfg])
# base_cfg = Config(inference_config).update([model_cfg, metrics_cfg])

cfgs = []
for cfg_update in dict_product(dataset_options):
    new_cfg = base_cfg.update(cfg_update)
    check_missing(new_cfg) # Verify there are no ? in config.
    cfgs.append(new_cfg)

In [9]:
len(cfgs)

4

In [10]:
cfgs[0]

Config({'log': {'root': '/storage/vbutoi/scratch/ESE/inference/01_27_24_CityScapes_Analysis/CityScapes_Individual_Uncalibrated', 'log_interval': 10, 'log_image_stats': True, 'log_pixel_stats': True, 'show_examples': False}, 'experiment': {'seed': 42}, 'data': {'split': 'cal', 'input_type': 'image', 'preload': False}, 'dataloader': {'batch_size': 1, 'num_workers': 0, 'pin_memory': False}, 'calibration': {'num_bins': 15, 'neighborhood_width': 3, 'square_diff': False}, 'model': {'pretrained_exp_root': '/storage/vbutoi/scratch/ESE/training/01_25_24_CityScapes_Dice/20240125_153459-4IQH-85df8c2ee8ce0787704fb7216d7a26df', 'checkpoint': 'max-val-dice_score', 'ensemble': False, 'ensemble_cfg': None, 'ensemble_w_metric': None, 'pretrained_select_metric': 'val-dice_score'}, 'qual_metrics': [{'Dice': {'_fn': 'ese.experiment.metrics.dice_score', 'from_logits': True, 'batch_reduction': 'mean', 'ignore_empty_labels': True, 'ignore_index': 0, 'metric_type': 'quality'}}, {'HD95': {'_fn': 'ese.experimen

## Running Jobs

In [11]:
from ese.experiment.experiment import run_ese_exp, submit_ese_exps
from ese.experiment.analysis.inference import get_cal_stats

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [12]:
###### Run individual jobs
run_ese_exp(
    config=cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    gpu='0',
) 

Set seed: 42


/storage/vbutoi/projects/ionpy/util/libcheck.py:49: UserWarning: Intel MKL extensions not available for NumPy
  warn("Intel MKL extensions not available for NumPy")
/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


Set seed: 42


KeyError: 'conf_interval'

In [ ]:
# ####### Run Batch Jobs
# submit_ese_exps(
#     exp_root=log_root,
#     job_func=get_cal_stats,
#     config_list=cfgs,
#     # available_gpus=['0', '1'],
#     # available_gpus=['2', '3'],
#     # available_gpus=['4', '5'],
#     # available_gpus=['6', '7'],
#     # available_gpus=['0', '1', '2']
#     # available_gpus=['1', '2', '3']
#     # available_gpus=['0', '1', '2', '3']
#     # available_gpus=['4', '5', '6', '7']
#     # available_gpus=['4', '5', '6']
#     # available_gpus=['5', '6', '7']
# )